In [3]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [56]:
events = utils.get_events()
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


In [57]:
devices = events[events['event'] == 'visited site'].groupby(['person', 'device_type']).agg({'timestamp':'count'}).reset_index()
devices_per_person = devices.pivot_table(index='person', columns='device_type', values='timestamp', aggfunc='sum', fill_value=0)
devices_per_person = devices_per_person.drop('Unknown', axis=1)
devices_per_person.head()        

device_type,Computer,Smartphone,Tablet
person,,,
0008ed71,2,0,0
00091926,34,0,0
00091a7a,0,1,0
000ba417,6,0,0
000c79fe,0,1,0


In [58]:
index_df = devices_per_person.index.to_frame(index=False)
devices_per_person = devices_per_person.reset_index(drop=True)
devices_per_person = pd.merge(index_df, devices_per_person, left_index=True, right_index=True)
devices_per_person.head()

,person,Computer,Smartphone,Tablet
0,0008ed71,2,0,0
1,00091926,34,0,0
2,00091a7a,0,1,0
3,000ba417,6,0,0
4,000c79fe,0,1,0


In [59]:
devices_per_person['most_used_device'] = devices_per_person[['Computer','Smartphone', 'Tablet']].idxmax(axis=1)
devices_per_person.head()

,person,Computer,Smartphone,Tablet,most_used_device
0,0008ed71,2,0,0,Computer
1,00091926,34,0,0,Computer
2,00091a7a,0,1,0,Smartphone
3,000ba417,6,0,0,Computer
4,000c79fe,0,1,0,Smartphone


In [64]:
#one hot encoding
devices_per_person = pd.concat([devices_per_person,pd.get_dummies(devices_per_person['most_used_device'], prefix='country')],axis=1)
devices_per_person = devices_per_person.drop('most_used_device', axis=1)
devices_per_person.head()

,person,Computer,Smartphone,Tablet,country_Computer,country_Smartphone,country_Tablet
0,0008ed71,2,0,0,1,0,0
1,00091926,34,0,0,1,0,0
2,00091a7a,0,1,0,0,1,0
3,000ba417,6,0,0,1,0,0
4,000c79fe,0,1,0,0,1,0


In [65]:
devices_per_person.to_csv('features_dispositivos.csv', index = False)